# Linear Regression Practice

In this lab we are going to use a 1994 dataset that has detailed prices on items sold at over 400 Burguer King, Wendy's, KFC and Roy Roger restaurants in New Jersey and Pensylvania.

Roy Roger https://en.wikipedia.org/wiki/Roy_Rogers_Restaurants


The data set has zip-code level data on various items prices, characterisitics of the zip code population.

The idea with this exercise is to see wheter fast-food restaurants charge higher prices in areas with a larger concentration of African Americans. 

<img src="fastfood.png" width=600px>

### Data Set Characteristics:  
K. Graddy (1997), "Do Fast-Food Chains Price Discriminate on the Race and Income Characteristics of an Area?" Journal of Business and Economic Statistics 15, 391-401.
http://people.brandeis.edu/~kgraddy/published%20papers/GraddyK_jbes1997.pdf

    :Number of Instances: 410
    
    :Attribute Information
    
    psoda         price of medium soda, 1st wave
    pfries        price of small fries, 1st wave
    pentree       price entree (burger or chicken), 1st wave
    wagest        starting wage, 1st wave
    nmgrs         number of managers, 1st wave
    nregs         number of registers, 1st wave
    hrsopen       hours open, 1st wave
    emp           number of employees, 1st wave
    psoda2        price of medium soday, 2nd wave
    pfries2       price of small fries, 2nd wave
    pentree2      price entree, 2nd wave
    wagest2       starting wage, 2nd wave
    nmgrs2        number of managers, 2nd wave
    nregs2        number of registers, 2nd wave
    hrsopen2      hours open, 2nd wave
    emp2          number of employees, 2nd wave
    compown       =1 if company owned
    chain         BK = 1, KFC = 2, Roy Rogers = 3, Wendy's = 4
    density       population density, town
    crmrte        crime rate, town
    state         NJ = 1, PA = 2
    prpblck       proportion black, zipcode
    prppov        proportion in poverty, zipcode
    prpncar       proportion no car, zipcode
    hseval        median housing value, zipcode
    nstores       number of stores, zipcode
    income        median family income, zipcode
    county        county label
    lpsoda        log(psoda)
    lpfries       log(pfries)
    lhseval       log(hseval)
    lincome       log(income)
    ldensity      log(density)
    NJ            =1 for New Jersey
    BK            =1 if Burger King
    KFC           =1 if Kentucky Fried Chicken
    RR            =1 if Roy Rogers


In [14]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Stats/Regresions Packages
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import patsy
import statsmodels.api as sm
from sklearn.metrics import r2_score

# make sure charts appear in the notebook:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

#### 1. Read the data, print the last 5 rows, perform EDA (missing values, data types, describe)

In [26]:
path_to_file = 'discrim.csv' ## Change it to your path
fast_food = pd.read_csv(path_to_file)
fast_food.tail(5)

,psoda,pfries,pentree,wagest,nmgrs,nregs,hrsopen,emp,psoda2,pfries2,...,county,lpsoda,lpfries,lhseval,lincome,ldensity,NJ,BK,KFC,RR
405,1.11,1.05,0.94,4.50,3.0,2.0,12.5,10.0,1.11,1.11,...,20,0.104360,0.048790,11.94601,10.90837,7.253400,0,0,0,1
406,0.95,0.74,2.33,4.75,3.0,4.0,10.0,8.0,0.95,0.84,...,20,-0.051293,-0.301105,11.49476,10.61115,8.368465,0,0,1,0
407,0.97,0.84,0.91,4.25,4.0,3.0,18.0,35.0,0.97,0.86,...,20,-0.030459,-0.174353,11.49476,10.61115,8.368465,0,1,0,0
408,0.97,0.86,0.89,4.75,4.0,3.0,17.0,18.5,0.97,0.84,...,20,-0.030459,-0.150823,11.49476,10.61115,8.368465,0,1,0,0
409,1.02,0.89,0.90,4.50,3.0,3.0,15.0,29.0,1.02,0.94,...,3,0.019803,-0.116534,11.68435,10.53148,6.114567,0,1,0,0


In [15]:
fast_food.columns

Index([u'psoda', u'pfries', u'pentree', u'wagest', u'nmgrs', u'nregs',
       u'hrsopen', u'emp', u'psoda2', u'pfries2', u'pentree2', u'wagest2',
       u'nmgrs2', u'nregs2', u'hrsopen2', u'emp2', u'compown', u'chain',
       u'density', u'crmrte', u'state', u'prpblck', u'prppov', u'prpncar',
       u'hseval', u'nstores', u'income', u'county', u'lpsoda', u'lpfries',
       u'lhseval', u'lincome', u'ldensity', u'NJ', u'BK', u'KFC', u'RR'],
      dtype='object')

In [27]:
fast_food['crmrte'] = fast_food['crmrte'].astype('category')
fast_food['county'] = fast_food['county'].astype('category')
fast_food['state'] = fast_food['state'].astype('category')
#fast_food['prpblck'] = fast_food['prpblck'].astype('category')
#fast_food['prppov'] = fast_food['prppov'].astype('category')

In [28]:
from sklearn.cross_validation import KFold
kf = KFold(len(fast_food), n_folds=10, shuffle=True, random_state=43)
kf

sklearn.cross_validation.KFold(n=410, n_folds=10, shuffle=True, random_state=43)

In [29]:
Rsq_10xCV = []

model_def = 'psoda ~ county + prppov'

for train_index, test_index in kf:
    train_df = fast_food.iloc[train_index]
    y_train, X_train = patsy.dmatrices(model_def, 
                                       data=fast_food, return_type='dataframe')
    model = sm.OLS(y_train, X_train)
    results = model.fit()
    test_df = fast_food.iloc[test_index]
    y_test, X_test = patsy.dmatrices(model_def, 
                                     data=fast_food, return_type='dataframe')
    y_predicted = results.predict(X_test)
    Rsq_10xCV.append(r2_score(y_predicted,y_test))

In [30]:
np.mean(Rsq_10xCV)

-1.0384501756097557